<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/LSTM_AAPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ta
import yfinance as yf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.linear_model import Lasso
import ta

# Download data
data = yf.download('MSFT', start='2016-01-01', end='2023-03-26')
if data.empty:
    raise ValueError("No data found for the specified dates.")
prices = data['Close'].to_numpy()

# Add technical indicators
rsi = ta.momentum.RSIIndicator(data['Close'], window=14)
data['RSI'] = rsi.rsi()

stoch = ta.momentum.StochasticOscillator(high=data['High'], low=data['Low'], close=data['Close'], window=14, smooth_window=3)
data['%K'] = stoch.stoch()

def predict(prices):
    # Split data into training and testing sets
    train_prices = prices[:-365]
    test_prices = prices[-365:]
    if len(train_prices) < 5:
        return ["hold"] * len(test_prices), 0

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(train_prices.reshape(-1, 1))

    # Add RSI and Stochastic Oscillator to training data
    train_rsi = data['RSI'][:-365].to_numpy().reshape(-1, 1)
    train_k_percent = data['%K'][:-365].to_numpy().reshape(-1, 1)
    X_train = np.concatenate((X_train, train_rsi, train_k_percent), axis=1)

    # Use grid search to find the best hyperparameters
    if len(train_prices) >= 5:
        param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
        grid_search = GridSearchCV(Lasso(), param_grid, cv=5)
        grid_search.fit(X_train[:-1], X_train[1:])
        best_alpha = grid_search.best_params_['alpha']
    else:
        best_alpha = 0.001

    # Train the model
    model = Sequential()
    model.add(LSTM(128, input_shape=(1, 3)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train[:-1].reshape(-1, 1, 3), X_train[1:], epochs=50, batch_size=1, verbose=0)

    # Make prediction
    X_test = scaler.transform(test_prices.reshape(-1, 1))
    test_rsi = data['RSI'][-365:].to_numpy().reshape(-1, 1)
    test_k_percent = data['%K'][-365:].to_numpy().reshape(-1, 1)
    X_test = np.concatenate((X_test, test_rsi, test_k_percent), axis=1)
    y_pred = model.predict(X_test.reshape(-1, 1, 3))
    y_pred = scaler.inverse_transform(y_pred).flatten()

    # Calculate daily returns
    daily_returns = np.diff(prices[-366:]) / prices[-366:-1]
    strategy_returns = np.diff(y_pred) / test_prices[:-1]

    # Calculate Sharpe ratio
    sharpe_ratio = (np.mean(strategy_returns) - np.mean(daily_returns)) / np.std(daily_returns)

    # Convert the outcome probabilities to "buy", "sell", or "hold"
    predictions = []
    for i in range(1, len(y_pred)):
        if y_pred[i] > y_pred[i-1] and k_percent[i] > d_percent[i]:
            predictions.append("buy")
        elif y_pred[i] < y_pred[i-1] and k_percent[i] < d_percent[i]:
            predictions.append("sell")
        else:
            predictions.append("hold")

    return predictions, sharpe_ratio


# Test the model
data = yf.download('AAPL', start='2022-03-16', end='2022-03-23')
if data.empty:
    raise ValueError("No data found for the specified dates.")
prices = data['Close'].to_numpy()
predictions, sharpe_ratio = predict(prices)
print("Trading strategy predictions:")
for i in range(len(predictions)):
 print(f"Day {i+1}: {predictions[i]}")
 print(f"Sharpe ratio: {sharpe_ratio}")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=95872665f73e53cc74c6f08bfe7e44c64764b809be4be537adccb2ee61b700fa
  Stored in directory: /root/.cache/pip/wheels/6a/76/03/7f785aaa50b9c6ec7e3fd105a62c1b2c45a034512d51e024a9
Successfully built ta
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Trading strategy predictions:
Day 1: hold
Sharpe ratio: 0
Day 2: hold
Sharpe ratio: 0
Day 3: hold
Sharpe ratio: 0
Day 4: hold
Sharpe ratio: 0
Day 5: hold
Sharpe ratio: 0
